In [1]:
!pip uninstall -y torch torchvision torchaudio


Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0


In [2]:
!pip install torch==2.2.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [3]:
!pip install torchtext==0.17.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 59.4 MB/s eta 0:00:0000:0100:01


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
import torch
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
df = pd.read_csv('/kaggle/input/language-translation-englishfrench/eng_-french.csv')
df.columns = ["eng","fre"]
df.head()

,eng,fre
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [7]:
print(df.isna().sum())
print(df.isnull().sum())

eng    0
fre    0
dtype: int64
eng    0
fre    0
dtype: int64


In [8]:
# tokenize

In [9]:
import string
import re

valid_puncs = "-,'!?."
invalid_puncs = string.punctuation         #returns all sets of punctuations

for punc in valid_puncs:
    invalid_puncs = invalid_puncs.replace(punc,"")
invalid_puncs_re = re.compile("["+ invalid_puncs +"]")   #expression to match any invalid punctuations

def text_sanitization(text: str) -> str:
    text = invalid_puncs_re.sub("", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [10]:
df["eng"] = df["eng"].apply(text_sanitization)
df["fre"] = df["fre"].apply(text_sanitization)
df.tail(5)

,eng,fre
175616,"Top-down economics never works, said Obama. Th...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
175620,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [11]:
import math
import random
import re
import string
from collections import Counter, OrderedDict
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
import os

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
en_words = Counter()
for sentence in tqdm(df["eng"]):
    en_words.update(word_tokenize(sentence, language='english'))
en_words = OrderedDict(sorted(en_words.items(), key=lambda x:-x[1]))

fr_words = Counter()
for sentence in tqdm(df["fre"]):
    fr_words.update(word_tokenize(sentence, language='french'))
fr_words = OrderedDict(sorted(fr_words.items(), key=lambda x:-x[1]))

print(f"no. of english words: {len(en_words)}")
print(f"no. of french words: {len(fr_words)}")

  0%|          | 0/175621 [00:00<?, ?it/s]

  0%|          | 0/175621 [00:00<?, ?it/s]

no. of english words: 16220
no. of french words: 31581


In [38]:
en_words['a']

24320

In [13]:
from sys import version
import torch
torch.__version__

'2.2.0+cu121'

In [14]:
import torchtext
from torchtext.vocab import vocab


In [15]:
unk_token = '<unk>'  # Unknown
sos_token = '<sos>'  # Start of sentence
eos_token = '<eos>'  # End of sentence
pad_token = '<pad>'  # Padding token
en_vocab = torchtext.vocab.vocab(en_words, specials=[unk_token, eos_token, pad_token])
en_vocab.set_default_index(en_vocab[unk_token])
fr_vocab = torchtext.vocab.vocab(fr_words, specials=[unk_token, eos_token, sos_token, pad_token])
fr_vocab.set_default_index(fr_vocab[unk_token])

In [16]:
len(en_vocab), len(fr_vocab)

(16223, 31585)

In [17]:
def en_tokenize(text: str, append_eos=True):
    words = word_tokenize(text, language='english')
    if append_eos:
        words.append(eos_token)
    return en_vocab(words)

def fr_tokenize(text: str, append_eos=True):
    words = [sos_token] + word_tokenize(text, language='french')
    if append_eos:
        words.append(eos_token)
    return fr_vocab(words)

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: torchtext._torchtext.Vocab, arg0: str) -> int

Invoked with: <torchtext._torchtext.Vocab object at 0x7c90a87f5a70>, 0

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df['eng'], df['fre'], test_size=0.2)

In [20]:
x_train_seqs = [en_tokenize(x) for x in x_train]
x_test_seqs = [en_tokenize(x) for x in x_test]

y_train_seqs = [fr_tokenize(y) for y in y_train]
y_test_seqs = [fr_tokenize(y) for y in y_test]

KeyboardInterrupt: 

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, src_sequences, tgt_sequences):
        self.src_sequences = src_sequences
        self.tgt_sequences = tgt_sequences

    def __len__(self):
        return len(self.src_sequences)

    def __getitem__(self, idx):
        src = self.src_sequences[idx]
        tgt = self.tgt_sequences[idx]
        return src, tgt

In [ ]:
def pad_collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)

    # Pad sequences
    src_padded = pad_sequence([torch.tensor(seq) for seq in src_batch], batch_first=True, padding_value=0)
    tgt_padded = pad_sequence([torch.tensor(seq) for seq in tgt_batch], batch_first=True, padding_value=0)

    # Create padding masks
    src_mask = (src_padded == 0).float()
    tgt_mask = (tgt_padded == 0).float()

    return src_padded, tgt_padded, src_mask, tgt_mask


In [ ]:
x_train_seqs = pad_sequence([torch.tensor(seq).to(device) for seq in x_train_seqs], batch_first=True, padding_value=2)
y_train_seqs = pad_sequence([torch.tensor(seq).to(device) for seq in y_train_seqs], batch_first=True, padding_value=3)

x_test_seqs = pad_sequence([torch.tensor(seq).to(device) for seq in x_test_seqs], batch_first=True, padding_value=2)
y_test_seqs = pad_sequence([torch.tensor(seq).to(device) for seq in y_test_seqs], batch_first=True, padding_value=3)

train_dataset = TranslationDataset(x_train_seqs, y_train_seqs)
test_dataset = TranslationDataset(x_test_seqs, y_test_seqs)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
input_dim = len(en_vocab)  # Vocabulary size for input
output_dim = len(fr_vocab)  # Vocabulary size for output
d_model = 256
nhead = 8
num_encoder_layers = 4
num_decoder_layers = 4
max_len = max(len(sentence) for sentence in x_train)
batch_size = 128
num_epochs = 25
dropout_val = 0.1
src_pad_index = 2
tgt_pad_index = 3

In [ ]:
max_len

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model).to(device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1).to(device)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)).to(device)
        pe[:, 0::2] = torch.sin(position * div_term).to(device)
        pe[:, 1::2] = torch.cos(position * div_term).to(device)
        pe = pe.unsqueeze(0).transpose(0, 1).to(device)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :x.size(1), :]
        return x


In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_encoder_layers, num_decoder_layers, output_dim, max_len, dropout_val):
        super(Transformer, self).__init__()
        self.model_dim = model_dim
        self.positional_encoding = PositionalEncoding(d_model=model_dim, max_len=max_len)
        self.transformer = nn.Transformer(d_model=model_dim, nhead=num_heads, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dropout=dropout_val, batch_first=True)
        self.input_embedding = nn.Embedding(input_dim, model_dim, padding_idx=src_pad_index)
        self.output_embedding = nn.Embedding(output_dim, model_dim, padding_idx=tgt_pad_index)
        self.layer_norm_src = nn.LayerNorm(d_model)
        self.layer_norm_tgt = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_val)
        self.fc_out = nn.Linear(model_dim, output_dim)
        
    def create_look_ahead_mask(self, size):
        mask = torch.triu(torch.ones(size, size, dtype=torch.float), diagonal=1)
        return mask.masked_fill(mask == 1, float('-inf'))  # Shape: [seq_len, seq_len]

    def forward(self, src, tgt, src_padding_mask=None, tgt_padding_mask=None):
        src = (self.input_embedding(src) * math.sqrt(self.model_dim)).to(device)
        tgt = (self.output_embedding(tgt) * math.sqrt(self.model_dim)).to(device)
        
        src = self.dropout(src).to(device)
        tgt = self.dropout(tgt).to(device)
        
        src = self.layer_norm_src(src)
        tgt = self.layer_norm_tgt(tgt)
#         print("done till here")
        src = self.positional_encoding(src).to(device)
        tgt = self.positional_encoding(tgt).to(device)
        tgt_look_ahead_mask = self.create_look_ahead_mask(tgt.size(1))  # Shape: [tgt_seq_len, tgt_seq_len]
        tgt_look_ahead_mask = tgt_look_ahead_mask.to(device)
#         print("done till here")
#         print(src.size(), tgt.size())
        output = self.transformer(src, tgt, 
                                  src_key_padding_mask=src_padding_mask.to(device), 
                                  tgt_key_padding_mask=tgt_padding_mask.to(device), 
                                  tgt_mask=tgt_look_ahead_mask).to(device)
        return self.fc_out(output).to(device)

In [ ]:
# input_dim = 10  # Vocabulary size
# model_dim = 512  # Dimension of model
# num_heads = 8
# num_layers = 6
# output_dim = 10  # Output dimension

model = Transformer(input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, output_dim, max_len, dropout_val).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = tgt_pad_index)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def save_model(model, epoch, directory='/kaggle/working/'):
    filename = (f"ep-{epoch+1}-.pth")
    filepath = os.path.join(directory, filename)
    
    torch.save(model.state_dict(), filepath)
    print(f"Model saved to {filepath}")

In [ ]:
for epoch in range(num_epochs):
    model.train()

    losses = 0.0
    correct_predictions = 0
    total_predictions = 0

    for src_batch, tgt_batch in tqdm(train_loader):
        optimizer.zero_grad()
        
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        src_padding_mask = ((src_batch == 2).bool()).to(device)
        tgt_padding_mask = (tgt_batch == 3).bool().to(device)

        output = model(src_batch, tgt_batch, src_padding_mask, tgt_padding_mask).to(device)

        loss = criterion(output.view(-1, output.size(-1)), tgt_batch.contiguous().view(-1))
        losses += loss.item()  # Use .item() to get the scalar value

        loss.backward()
        optimizer.step()
        
        preds = output.argmax(dim=-1)  
        correct_predictions += (preds == tgt_batch).sum().item()
        total_predictions += tgt_batch.numel()
    
    avg_epoch_loss = losses / len(train_loader)
    accuracy = correct_predictions / total_predictions

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_epoch_loss}, Accuracy: {accuracy}')
    save_model(model, epoch)
    print(f'Model saved at {epoch+1}')

In [ ]:
model.eval()

# Initialize metrics
total_loss = 0.0
correct_predictions = 0
total_predictions = 0

# Disable gradient calculations for evaluation
with torch.no_grad():
    for src_batch, tgt_batch in tqdm(test_loader):
        src_batch = src_batch.to(device)
        tgt_batch = tgt_batch.to(device)

        src_padding_mask = (src_batch == 0).to(dtype=torch.bool, device=device)
        tgt_padding_mask = (tgt_batch == 0).to(dtype=torch.bool, device=device)

        # Forward pass
        output = model(src_batch, tgt_batch, src_padding_mask, tgt_padding_mask)

        # Compute loss
        loss = criterion(output.view(-1, output.size(-1)), tgt_batch.contiguous().view(-1))
        total_loss += loss.item()  # Use .item() to get the scalar value

        # Compute predictions
        preds = output.argmax(dim=-1)

        # Calculate correct predictions
        correct_predictions += (preds == tgt_batch).sum().item()
        total_predictions += tgt_batch.numel()

# Calculate average loss and accuracy
avg_loss = total_loss / len(test_loader)
accuracy = correct_predictions / total_predictions

# Print evaluation results
print(f'Loss: {avg_loss}, Accuracy: {accuracy}')

In [ ]:
en_vocab[0]